# Procedure

Iterate through the file names of the images.

Extract the PWM values from each file name. The extraction process will depend on the specific format and structure of the file names. 

Associate each PWM value with its corresponding image and quality number. need to read the image data and quality number from the files or a separate data source.

Create a dataset that includes the PWM values, image data, and quality numbers. This dataset will serve as the input for the Bayesian optimization algorithm.

Split the dataset into a training set and a validation set. The training set will be used to train the surrogate model, while the validation set will be used to assess the performance and tune the hyperparameters of the Bayesian optimization algorithm.
Implement the objective function, which takes the PWM values as inputs and returns the quality number. use existing image data and quality numbers to compute the quality number for a given set of PWM values.

Choose an appropriate acquisition function, such as Expected Improvement (EI) or Upper Confidence Bound (UCB), to guide the Bayesian optimization search. (more to adjust later)
Initialize the Bayesian optimization algorithm with the training data, objective function, and acquisition function.

Run the Bayesian optimization loop, which suggests new sets of PWM values to evaluate based on the surrogate model and the acquisition function. Evaluate each suggested set of PWM values, compute the corresponding quality number using the objective function, and update the surrogate model with the new data.

Repeat the optimization loop for a predefined number of iterations or until a satisfactory solution is found. Monitor the progress and convergence of the algorithm by tracking the best observed PWM values and corresponding quality numbers.

In [6]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
# from skopt import BayesSearchCV

# Step 1: Specify the directory path where the files are located
directory = "/home/zw3721/Downloads/2022_master_project/M5"  # Replace with the actual directory path

# Step 2: Get the file names from the directory
file_names = os.listdir(directory)

# Step 3: Extract PWM values from file names
pwm_values = []
for file_name in file_names:
    pwm_string = file_name.split("pwm=")[1].split("-")[:5]  # Extract the PWM values as strings
    pwm_values.append([int(value) for value in pwm_string])  # Convert PWM values to integers

In [7]:
import glob
import PIL
import matplotlib.pyplot as plt 
import numpy as np
from scipy import stats
from PIL import Image
from scipy import optimize
from sklearn.metrics import r2_score
import pandas as pd
from matplotlib.patches import Rectangle
from pylab import gray, imshow, show
import cv2
import imutils
import laserbeamsize as lbs

image_list = glob.glob(r'/home/zw3721/Downloads/2022_master_project/M5/*.png')

score = []
for i in range (len(image_list)):
    img = Image.open(image_list[i])
    img = img.convert('L')
    image_array = np.array(img)
    x, y, dx, dy, phi = lbs.beam_size(image_array)
    Ellipticity = min(dx,dy)/max(dx,dy) 
    threshold_up_area = (image_array >= np.max(image_array)/10)+0   
    total_pixel = image_array.shape[0]*image_array.shape[1]   
    dispersion = total_pixel/np.sum(threshold_up_area) 
    quality_number = dispersion*Ellipticity 
    score.append(quality_number)
    
quality_numbers = score  # List of corresponding quality numbers

In [ ]:
img_data_array=[]

for i in range (len(image_list)):

    img=cv2.imread(image_list[i])
    image_array = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    image_array=np.array(image_array)
    image_array = image_array.astype('float32')
    image_array /= 255 
    img_data_array=np.append(img_data_array,image_array)

In [ ]:
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from skopt import BayesSearchCV

# Step 3: Create dataset
dataset = np.column_stack((pwm_values, image_data, quality_numbers))

# Step 4: Split dataset into training and validation sets
train_data, val_data = train_test_split(dataset, test_size=0.2, random_state=42)

# Step 5: Define objective function
def objective_function(pwm_values):
    # Use your image data and quality numbers to compute the quality number for a given set of PWM values
    # You may need to write your own implementation here based on your specific computation
    # Return the computed quality number
    return quality_number

# Step 6: Define the search space for Bayesian optimization
param_space = { f'pwm_{i}': (0, 4095) for i in range(5) }  # Adjust the range of PWM values as per your requirements

# Step 7: Initialize Bayesian optimization
bayes_optimizer = BayesSearchCV(
    estimator=None,  # You can specify your own estimator if needed
    search_spaces=param_space,
    n_iter=100,  # Adjust the number of iterations as per your requirements
    n_jobs=-1
)

# Step 8: Run Bayesian optimization
bayes_optimizer.fit(train_data[:, :-1], train_data[:, -1])

# Step 9: Get the best PWM values found by Bayesian optimization
best_pwm_values = [bayes_optimizer.best_params_[f'pwm_{i}'] for i in range(5)]

# Step 10: Evaluate the quality number using the best PWM values
best_quality_number = objective_function(best_pwm_values)

print("Best PWM values:", best_pwm_values)
print("Corresponding quality number:", best_quality_number)